In [1]:
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
mp_pose = mp.solutions.pose

In [3]:
# Define landmark points
landmarks_points = {
    "nose": 0,
    "left_shoulder": 11, "right_shoulder": 12,
    "left_elbow": 13, "right_elbow": 14,
    "left_wrist": 15, "right_wrist": 16,
    "left_hip": 23, "right_hip": 24,
    "left_knee": 25, "right_knee": 26,
    "left_ankle": 27, "right_ankle": 28,
    "left_heel": 29, "right_heel": 30,
    "left_foot_index": 31, "right_foot_index": 32,
}

# Generate column names
col_names = []
for name in landmarks_points.keys():
    col_names.extend([f"{name}_x", f"{name}_y", f"{name}_z", f"{name}_v"])

pose_name = col_names + ["pose"]

In [ ]:
# Initialize lists
pose_list = []
main_dir = "./poses_dataset/Images"
pose_dir_list = [d for d in os.listdir(main_dir) if os.path.isdir(os.path.join(main_dir, d))]

# Process directories
for i, pose_dir in enumerate(pose_dir_list):
    images_dir_list = os.listdir(f"{main_dir}/{pose_dir}")
    for image_file in images_dir_list:
        pre_list = []
        image_path = f"{main_dir}/{pose_dir}/{image_file}"
        with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
            image = cv2.imread(image_path)
            if image is None:
                print(f"Could not read image: {image_path}")
                continue
            result = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            try:
                for landmarks in result.pose_landmarks.landmark:
                    pre_list.append(landmarks)
            except AttributeError:
                print(f"No pose landmarks detected in {image_path}")
                continue

        # If landmarks were detected
        if pre_list:
            gen1116 = np.array([
                [pre_list[m].x, pre_list[m].y, pre_list[m].z, pre_list[m].visibility]
                for m in range(11, 17)
            ]).flatten().tolist()

            gen2333 = np.array([
                [pre_list[m].x, pre_list[m].y, pre_list[m].z, pre_list[m].visibility]
                for m in range(23, 33)
            ]).flatten().tolist()

            all_list = [
                pre_list[0].x, pre_list[0].y, pre_list[0].z, pre_list[0].visibility
            ]
            all_list.extend(gen1116 + gen2333)
            all_list.append(i)

            pose_list.append(all_list)

# Save the data to a CSV
data_pose = pd.DataFrame(pose_list, columns=pose_name)
os.makedirs("./csv_files", exist_ok=True)
data_pose.to_csv("./csv_files/poses_data_pose.csv", index=False)
print("Pose data saved to ./csv_files/poses_data_pose.csv")

No pose landmarks detected in ./poses_dataset/Images/goddess/46.jpg
